In [31]:
import numpy as np 
from PIL import Image
import pathlib
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader
import torchvision
import torchvision.transforms as transforms

class MaskClassificationCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            # Layer 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),  # Output: 32 x 64 x 64
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),  # Output: 64 x 64 x 64
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Output: 64 x 32 x 32
            nn.Dropout(0.4),

            # Layer 2
            nn.Conv2d(64, 128, kernel_size=3, padding=1),  # Output: 128 x 32 x 32
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),  # Output: 128 x 32 x 32
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Output: 128 x 16 x 16
            nn.Dropout(0.6),

            # Layer 3
            nn.Conv2d(128, 256, kernel_size=3, padding=1),  # Output: 256 x 16 x 16
            nn.ReLU(),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),  # Output: 256 x 16 x 16
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # Output: 256 x 8 x 8
            nn.Dropout(0.6),

            # Flatten and Fully Connected Layers
            nn.Flatten(),
            nn.Linear(256 * 16 * 16, 1024),  # Input size for Linear layer
            nn.ReLU(),
            nn.Dropout(0.6),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 3)  # Output: 3 classes
        )

    def forward(self, xb):
        for layer in self.network:
            xb = layer(xb)
        return xb

root = pathlib.Path('Dataset/')
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

def prediction(img_path,transformer):
    
    image = Image.open(img_path).convert('RGB')
    image_tensor = transformer(image).float()
    image_tensor = image_tensor.unsqueeze_(0)
    
    if torch.cuda.is_available():
        image_tensor = image_tensor.cuda()
        
    input = Variable(image_tensor)
    output = model(input)
    inde = output.data.numpy().argmax()
    pred = classes[index]
    return pred

In [32]:
from PIL import Image
import torchvision.transforms as transforms

base_transforms = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1, hue=0.05),  # Mild color changes
    transforms.RandomAffine(degrees=0, translate=(0.1, 0.1)),  # Small translations (no rotation)
    transforms.RandomResizedCrop(128, scale=(0.9, 1.1), ratio=(0.9, 1.1)),  # Random cropping and scaling
    transforms.RandomApply([transforms.GaussianBlur(kernel_size=3)], p=0.2),  # Slight blur occasionally
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

incorrect_mask_augmentations = transforms.Compose([
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.GaussianBlur(3, sigma=(0.1, 2.0)),
])

initial_size=(128, 128)
final_size=(128, 128)

class CustomAugmentation:
    def __init__(self, incorrect_mask_augmentations, initial_size, final_size, base_transforms, incorrect_class_idx=0):
        self.incorrect_mask_augmentations = incorrect_mask_augmentations
        self.incorrect_class_idx = incorrect_class_idx
        self.base_transforms = base_transforms
        self.resize_size = initial_size

    def update_resize(self, final_size):
        self.resize_size = final_size

    def __call__(self, img, label):
        # Convert to PIL Image if img is a tensor
        if isinstance(img, torch.Tensor):
            img = transforms.ToPILImage()(img)

        # Apply resizing to the current size
        img = transforms.Resize(self.resize_size)(img)

        # Apply specific augmentations for "incorrectly worn mask" class
        if label == self.incorrect_class_idx:
            img = self.incorrect_mask_augmentations(img)

        # Apply base transforms and convert to tensor
        img = self.base_transforms(img)        
        return img

# Initialize CustomAugmentation with initial resizing
transform = CustomAugmentation(
    incorrect_mask_augmentations=incorrect_mask_augmentations,
    initial_size=(128, 128),
    final_size=(128, 128),
    base_transforms=base_transforms,
    incorrect_class_idx=0
)

In [33]:
data_set_dir = "Dataset" # loading training data

dataset = torchvision.datasets.ImageFolder(data_set_dir, transform=base_transforms)

test_len = 982
train_len = 8000
train_data, test_data = random_split(dataset, [train_len, test_len])

train_dataLoader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataLoader = DataLoader(train_data, batch_size=32)

In [34]:
import torch.optim as optim

# Use CUDA if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class_weights = torch.tensor([3.0, 1.0, 1.0], device=device) 
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

model = MaskClassificationCNN()

# Move the model to the appropriate device if using GPU
model = model.to(device)

# Training parameters
num_of_epoch = 15
learning_rate = 0.001
weight_decay = 1e-4
patience = 4

# Initialize the optimizer with weight decay
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

# Learning rate scheduler to reduce the learning rate when loss is plateauing
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=4, factor=0.5)

history = [] 
best_val_loss = float('inf')  
epochs_without_improvement = 0  

for epoch in range(num_of_epoch):
    model.train()  # Set model to training mode
    train_loss = 0.0

    for batch in train_dataLoader:
        images, labels = batch
        images, labels = images.to(device), labels.to(device)  # Move data to the device

        optimizer.zero_grad()

        outputs = model(images)  # Forward pass
        loss = loss_fn(outputs, labels)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_dataLoader)
    print(f"Epoch [{epoch+1}/{num_of_epoch}], Loss: {avg_train_loss:.4f}")

    model.eval()  # Set model to evaluation mode
    correct = 0
    total = 0
    val_loss = 0.0

    with torch.no_grad():
        for batch in test_dataLoader:
            images, labels = batch
            images, labels = images.to(device), labels.to(device)  # Move data to the device

            outputs = model(images)
            loss = loss_fn(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_val_loss = val_loss / len(test_dataLoader)
    # Adjust learning rate
    scheduler.step(avg_val_loss)
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%, Avg_Test_Loss: {avg_val_loss:.5f}")

    # Check if the validation loss improved for early stopping
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss 
        epochs_without_improvement = 0  
        # Save model whenever the best loss improves
        scripted_model = torch.jit.script(model)
        scripted_model.save("model_v3_scripted.pth")
    else:
        epochs_without_improvement += 1

    # Check for early stopping
    if epochs_without_improvement >= patience:
        print(f"Early stopping triggered after {epoch + 1} epochs.")
        break

    # Log training history
    history.append({'epoch': epoch+1, 'train_loss': avg_train_loss, 'test_accuracy': accuracy})

print("Training complete.")

Epoch [1/15], Loss: 0.6240
Test Accuracy: 76.33%, Avg_Test_Loss: 0.44624
Epoch [2/15], Loss: 0.4375
Test Accuracy: 77.97%, Avg_Test_Loss: 0.42447
Epoch [3/15], Loss: 0.3702
Test Accuracy: 83.79%, Avg_Test_Loss: 0.33800
Epoch [4/15], Loss: 0.3190
Test Accuracy: 89.75%, Avg_Test_Loss: 0.23315
Epoch [5/15], Loss: 0.2757
Test Accuracy: 86.59%, Avg_Test_Loss: 0.23259
Epoch [6/15], Loss: 0.2713
Test Accuracy: 90.75%, Avg_Test_Loss: 0.20075
Epoch [7/15], Loss: 0.2401
Test Accuracy: 90.88%, Avg_Test_Loss: 0.18635
Epoch [8/15], Loss: 0.2207
Test Accuracy: 93.29%, Avg_Test_Loss: 0.15068
Epoch [9/15], Loss: 0.2274
Test Accuracy: 93.31%, Avg_Test_Loss: 0.15687
Epoch [10/15], Loss: 0.2071
Test Accuracy: 93.36%, Avg_Test_Loss: 0.13733
Epoch [11/15], Loss: 0.1917
Test Accuracy: 94.15%, Avg_Test_Loss: 0.14771
Epoch [12/15], Loss: 0.1930
Test Accuracy: 92.12%, Avg_Test_Loss: 0.15029
Epoch [13/15], Loss: 0.1764
Test Accuracy: 94.94%, Avg_Test_Loss: 0.11263
Epoch [14/15], Loss: 0.1552
Test Accuracy: 94.9